# **Algoritmo de Clasificación de Bayes Ingenuo**

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn import datasets



Extracción de datos



In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
X = pd.read_csv(url, header = None)
y = X.iloc[:,0]
X.drop(X.columns[0], axis='columns', inplace=True)
X.columns = range(X.shape[1])
print(X)
print(y)

        0     1     2     3    4     5   ...    7     8      9     10    11    12
0    14.23  1.71  2.43  15.6  127  2.80  ...  0.28  2.29   5.64  1.04  3.92  1065
1    13.20  1.78  2.14  11.2  100  2.65  ...  0.26  1.28   4.38  1.05  3.40  1050
2    13.16  2.36  2.67  18.6  101  2.80  ...  0.30  2.81   5.68  1.03  3.17  1185
3    14.37  1.95  2.50  16.8  113  3.85  ...  0.24  2.18   7.80  0.86  3.45  1480
4    13.24  2.59  2.87  21.0  118  2.80  ...  0.39  1.82   4.32  1.04  2.93   735
..     ...   ...   ...   ...  ...   ...  ...   ...   ...    ...   ...   ...   ...
173  13.71  5.65  2.45  20.5   95  1.68  ...  0.52  1.06   7.70  0.64  1.74   740
174  13.40  3.91  2.48  23.0  102  1.80  ...  0.43  1.41   7.30  0.70  1.56   750
175  13.27  4.28  2.26  20.0  120  1.59  ...  0.43  1.35  10.20  0.59  1.56   835
176  13.17  2.59  2.37  20.0  120  1.65  ...  0.53  1.46   9.30  0.60  1.62   840
177  14.13  4.10  2.74  24.5   96  2.05  ...  0.56  1.35   9.20  0.61  1.60   560

[178 rows x 13 

El número de etiquetas en el dataset son 3

In [3]:
totalLabels = 3

Defina la proporción de datos para el conjunto de entrenamiento

In [4]:
prop = float(input("Proporcion de datos para el conjunto de entrenamiento: p = ")) # proporcion del conjunto de entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=prop) #, random_state=0)

Proporcion de datos para el conjunto de entrenamiento: p = 0.6


Función para calcular la media y la desviación estándar por cada característica de cada clase. Se asume que hay un número de $totalLabels$ de diferentes clases en el conjunto de entrenamiento $(X,y)$. También se asume que las clases son $1, 2, \ldots, totalLabels$.

In [5]:
def estadisticos(X, y, numElementos):
  # Funcion para calcular la media y la desviacion estandar por cada caracteristica de cada clase
  # numElementos es un arreglo tal que su entrada k es el numero de patrones de la clase k+1

  totalLabels = len(numElementos)

  m = len(y) 
  n = X.shape[1] # numero de caracteristicas

  sigma = np.zeros((totalLabels, n)) # Matriz cuya entrada (k,j) sera la desviacion estandar de la j-esima caracteristica en la k-esima clase
  mu = np.zeros((totalLabels, n)) # Matriz cuya entrada (k,j) sera la media de la j-esima caracteristica en la k-esima clase
  mu_2 = np.zeros((totalLabels, n)) # Matriz cuya entrada (k,j) sera el segundo momento de la j-esima caracteristica en la k-esima clase

  # Recordemos mu = (1/N)\sum_i x_i, mu_2 = (1/N)\sum_i x_i^2, y sigma = \sqrt{mu_2 - (mu)^2}.

  for i in range(m):
    k = int(y[i])-1
    
    for t in range(n):
      mu[k][t] += X[i][t]
      mu_2[k][t] += X[i][t]**2

  for k in range(totalLabels):
    for t in range(n):
      mu[k][t] = mu[k][t]/numElementos[k]
      mu_2[k][t] = mu_2[k][t]/numElementos[k]

      sigma[k][t] = (mu_2[k][t]-mu[k][t]**2)**(0.5)

  return mu, sigma


In [6]:
print(X_train.to_numpy())

[[1.438e+01 3.590e+00 2.280e+00 ... 1.040e+00 3.440e+00 1.065e+03]
 [1.323e+01 3.300e+00 2.280e+00 ... 5.600e-01 1.510e+00 6.750e+02]
 [1.311e+01 1.010e+00 1.700e+00 ... 1.120e+00 3.180e+00 5.020e+02]
 ...
 [1.200e+01 3.430e+00 2.000e+00 ... 9.300e-01 3.050e+00 5.640e+02]
 [1.208e+01 1.330e+00 2.300e+00 ... 1.070e+00 3.210e+00 6.250e+02]
 [1.242e+01 2.550e+00 2.270e+00 ... 8.600e-01 3.300e+00 3.150e+02]]


Función de densidad Gaussiana 
$$f(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$$

In [7]:
def gaussianDensity(x, mu, sigma): 

  e = math.exp(-(1/2)*(((x-mu)/sigma)**2))
  d = sigma*((2*math.pi)**(0.5))

  return e/d

Función de verosimilitud

$$\mathbb{P}(Y=C_k | X) = \frac{\mathbb{P}(Y=C_k)\prod_{j}\mathbb{P}(X_j | Y=C_k)}{\sum_i \mathbb{P}(Y=C_i)\prod_j \prod_i \mathbb{P}(X_j | Y=C_i) } $$


In [8]:
def likelihood(x, mu, sigma, numElementos):
  # x es el vector de caracteristicas
  # numElementos es un arreglo tal que numElementos[j] es el numero de patrones de la clase j+1

  n = len(x) # numero de caracteristicas
  totalLabels = len(numElementos) # numero de clases
  m = np.sum(numElementos) # numero total de patrones de entrenamiento

  p = np.zeros(totalLabels) # Arreglo que tendra p[k] = P(Y=C_k | X=x)

  for k in range(totalLabels):
    p[k] = numElementos[k]/m

    for j in range(n):
      p[k] = p[k]*gaussianDensity(x[j], mu[k][j], sigma[k][j])

  denom = np.sum(p)

  for k in range(totalLabels):
    p[k] = p[k]/denom

  return p

Función para obtener el máximo
$$ \hat{Y} = argmax_{C_k} P(Y=C_k | X=x)$$

In [9]:
def getMax(p):
  max = 0
  totalLabels = len(p)

  for i in range(1, totalLabels):
    if p[i]>p[max]:
      max = i

  return max+1

### Clase del clasificador Naive Bayes
Con métodos fit y predict

In [10]:
class NaiveBayes:
  def fit(self, X_train, y_train, totalLabels):
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()

    self.numElementos = np.zeros(totalLabels)
    m = len(y_train) # numero de patrones de entrenamiento
    n = X_train.shape[1] # numero de caracteristicas

    for i in range(m):
      k = int(y_train[i])-1
      self.numElementos[k] += 1

    self.mu, self.sigma = estadisticos(X_train, y_train, self.numElementos)

  def predict(self, X_test):
    X_test = X_test.to_numpy()
    m = X_test.shape[0]

    labels = []

    for i in range(m):
      p = likelihood(X_test[i], self.mu, self.sigma, self.numElementos)
      labels.append(getMax(p))
    
    return pd.Series(labels)


Función que reporta la matriz de confusión $M$, tal que $M[i][j]$ es la cantidad de patrones de la clase $i$ clasificados como clase $j$. También reporta el *accuracy*.

In [11]:
def performance(labels_pred, labels_real, totalLabels):
  # Reporta la matriz de confusion y la exactitud
  # labels_pred es el etiquetado hecho por el clasificador, labels_real son las etiquetas reales, y totalLabels es el numero total de etiquetas
  # Suponemos que las etiquetas son 1, 2, ..., totalLabels

  labels_pred = labels_pred.to_numpy()
  labels_real = labels_real.to_numpy()

  M = np.zeros((totalLabels, totalLabels))
  n = len(labels_pred)

  for i in range(n):
    M[int(labels_real[i])-1][int(labels_pred[i])-1] += 1

  acc = 0

  for i in range(totalLabels):
    acc += M[i][i]

  acc = acc/n
  M = pd.DataFrame(M, columns= range(1, totalLabels+1), index = range(1, totalLabels+1))

  return M, acc

Clasificación del conjunto de entrenamiento. Impresión de las etiquetas predichas, matriz de confunsión y exactitud.



In [12]:
classifier = NaiveBayes()
classifier.fit(X_train, y_train, totalLabels)
labels_train = classifier.predict(X_train)
M_train, acc_train = performance(labels_train, y_train, totalLabels)
print(labels_train)
print(M_train)
print(acc_train)

0      1
1      3
2      2
3      2
4      2
      ..
101    1
102    1
103    2
104    2
105    2
Length: 106, dtype: int64
      1     2     3
1  34.0   0.0   0.0
2   0.0  42.0   0.0
3   0.0   0.0  30.0
1.0


Clasificación del conjunto de prueba. Impresión de las etiquetas predichas, matriz de confunsión y exactitud.

In [13]:
labels_test = classifier.predict(X_test)
M_test, acc_test = performance(labels_test, y_test, totalLabels)
print(labels_test)
print(M_test)
print(acc_test)

0     1
1     1
2     2
3     3
4     2
     ..
67    3
68    1
69    2
70    2
71    3
Length: 72, dtype: int64
      1     2     3
1  24.0   1.0   0.0
2   0.0  28.0   1.0
3   0.0   0.0  18.0
0.9722222222222222
